In [1]:
import pandas as pd
import glob
import re
import matplotlib.pyplot as plt
import seaborn as sns
from zipfile import ZipFile

In [2]:
def init_state_social_distancing_actions(path):
  df_master = pd.DataFrame()
  flag = False
  files_in_folder = glob.glob(path)
  for filename in files_in_folder:    
    zip_file = ZipFile(filename)
    for text_file in zip_file.infolist():
      # if not text_file.filename.startswith('__MACOSX/'):
      if text_file.filename.endswith('.csv'):
        date = re.search('\d*-\d*-\d*', text_file.filename)[0]
        if date == "20201-06-01":
          date = "2021-06-01"
        date_time_value = pd.to_datetime(date)
        df = pd.read_csv(zip_file.open(text_file.filename), sep=",", header=0)
        df["Date"] = date_time_value
        df.rename(columns = {'Unnamed: 0':'State'}, inplace = True)
        df.drop((df[df.State.isin(["United States"])].index) | (df[df.State.isnull()].index), inplace=True)
      if not flag:
        df_master = df
        flag = True
      else:
        df_master = pd.concat([df_master, df])
  df_master.set_index(["Date", "State"], inplace=True)
  df_master.sort_index(inplace=True)
  return df_master

state_social_distancing_actions = init_state_social_distancing_actions(r'**csv_files/state_social_distancing_actions.zip')

In [3]:
def clean_state_social_distancing_actions(df):
  df = df.drop(columns=["Primary Election Postponement"])
  return df

cleaned_state_social_distancing_actions = clean_state_social_distancing_actions(state_social_distancing_actions)
cleaned_state_social_distancing_actions

State Is Easing Social Distancing Measures  \
Date       State                                                      
2020-06-04 Alabama                                              Yes   
           Alaska                                               Yes   
           Arizona                                              Yes   
           Arkansas                                             Yes   
           California                                           Yes   
...                                                             ...   
2021-11-09 Virginia                                             NaN   
           Washington                                           NaN   
           West Virginia                                        NaN   
           Wisconsin                                            NaN   
           Wyoming                                              NaN   

                         Stay at Home Order  \
Date       State                              
2020-06-04 Alabama                   Lifted   
           Alaska                    Lifted   
           Arizona                   Lifted   
           Arkansas                       -   
           California             Statewide   
...                                     ...   
2021-11-09 Virginia                     NaN   
           Washington                   NaN   
           West Virginia                NaN   
           Wisconsin                    NaN   
           Wyoming                      NaN   

                         Mandatory Quarantine for Travelers  \
Date       State                                              
2020-06-04 Alabama                                        -   
           Alaska                             All Travelers   
           Arizona                                   Lifted   
           Arkansas                     From Certain States   
           California                                     -   
...                                                     ...   
2021-11-09 Virginia                                     NaN   
           Washington                                   NaN   
           West Virginia                                NaN   
           Wisconsin                                    NaN   
           Wyoming                                      NaN   

                                            Non-Essential Business Closures  \
Date       State                                                              
2020-06-04 Alabama        All Non-Essential Businesses Permitted to Reop...   
           Alaska          All Non-Essential Businesses Permitted to Reopen   
           Arizona        All Non-Essential Businesses Permitted to Reop...   
           Arkansas                                                       -   
           California     Some Non-Essential Businesses Permitted to Reo...   
...                                                                     ...   
2021-11-09 Virginia                                                     NaN   
           Washington                                                   NaN   
           West Virginia                                                NaN   
           Wisconsin                                                    NaN   
           Wyoming                                                      NaN   

                               Large Gatherings Ban  \
Date       State                                      
2020-06-04 Alabama                           Lifted   
           Alaska                            Lifted   
           Arizona                           Lifted   
           Arkansas           >10 People Prohibited   
           California     All Gatherings Prohibited   
...                                             ...   
2021-11-09 Virginia                             NaN   
           Washington                           NaN   
           West Virginia                        NaN   
           Wisconsin                

Clean up the dataframe to remove unused columns and solve for Nan fields. 
- Have to manually insert face mask requirements for recent months based on: https://statepolicies.com/data/graphs/face-masks/
- Face mask mandate was reintroduced from Dec 15, 2021 -> Feb 15, 2022

In [4]:
california_data = cleaned_state_social_distancing_actions[cleaned_state_social_distancing_actions.index.get_level_values('State').isin(['California'])]
california_data.reset_index("State", inplace=True)
california_data = california_data[~california_data.index.duplicated(keep='first')]
# Fill inn missing dates with rows equal the previous date with data
days = pd.date_range(start=california_data.index[0], end=california_data.index[-1], freq="D")
california_data = california_data.asfreq('D', method="pad")

california_data
# TODO: Recreate removed graphs

,State,State Is Easing Social Distancing Measures,Stay at Home Order,Mandatory Quarantine for Travelers,Non-Essential Business Closures,Large Gatherings Ban,School Closures,Restaurant Limits,Emergency Declaration,Face Covering Requirement,Status of Reopening,Bar Closures,Bar Closures*,Statewide Face Mask Requirement
Date,,,,,,,,,,,,,,
2020-06-04,California,Yes,Statewide,-,Some Non-Essential Businesses Permitted to Reo...,All Gatherings Prohibited,Recommended Closure for School Year,Closed Except for Takeout/Delivery,Yes,NaN,NaN,NaN,NaN,NaN
2020-06-05,California,Yes,Statewide,-,Some Non-Essential Businesses Permitted to Reo...,All Gatherings Prohibited,Recommended Closure for School Year,Closed Except for Takeout/Delivery,Yes,NaN,NaN,NaN,NaN,NaN
2020-06-06,California,Yes,Statewide,-,Some Non-Essential Businesses Permitted to Reo...,All Gatherings Prohibited,Recommended Closure for School Year,Closed Except for Takeout/Delivery,Yes,NaN,NaN,NaN,NaN,NaN
2020-06-07,California,Yes,Statewide,-,Some Non-Essential Businesses Permitted to Reo...,All Gatherings Prohibited,Recommended Closure for School Year,Closed Except for Takeout/Delivery,Yes,NaN,NaN,NaN,NaN,NaN
2020-06-08,California,Yes,Statewide,-,Some Non-Essential Businesses Permitted to Reo...,All Gatherings Prohibited,Recommended Closure for School Year,Closed Except for Takeout/Delivery,Yes,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-05,California,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Reopened,NaN,NaN,Unvaccinated People Only
2021-11-06,California,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Reopened,NaN,NaN,Unvaccinated People Only
2021-11-07,California,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Reopened,NaN,NaN,Unvaccinated People Only
